
# Binomial Tree with **Greeks on Grids** + Control Variate (Tight In-Place Version)

**What this notebook does**
- Uses *Greeks on Grids* logic at the start of the tree: central differences over the first three nodes (levels 0–2).
- Prices **European and American** options simultaneously on a CRR tree.
- Computes **Δ, Γ** from levels 1 and 2 using an averaged grid spacing \(h=\tfrac12 S_0(u-d)\); forward **θ** via level-2 center.
- Applies **control variate** (CV) to American **value, Δ, Γ** using BS European as anchor.
- Reproduces the study: \(S_0=50,\ T=0.1,\ r=0.1,\ q=0.02,\ \sigma=0.3\), **puts** with \(K\in\{46,53\}\).
- Builds proxy with **1000 steps** (CV outputs) and finds the smallest \(N\) where CV meets the **average big-tree error** (uncorrected) from \(N\in\{100,105,110,120\}\).

In [ ]:

import math
from dataclasses import dataclass
from typing import Literal

import numpy as np
import pandas as pd
from scipy.stats import norm


## Black–Scholes (European) value, Δ, Γ

In [ ]:

OptionType = Literal["call","put"]

def _bs_d1_d2(S, K, r, q, sigma, T):
    if T <= 0:
        raise ValueError("T must be positive for BS Greeks.")
    sqrtT = math.sqrt(T)
    d1 = (math.log(S/K) + (r - q + 0.5*sigma*sigma)*T) / (sigma*sqrtT)
    d2 = d1 - sigma*sqrtT
    return d1, d2

def bs_value_delta_gamma(option: OptionType, S, K, r, q, sigma, T):
    if T <= 0:
        payoff = max(S-K,0.0) if option=="call" else max(K-S,0.0)
        if option=="call":
            delta = 1.0 if S>K else 0.0
        else:
            delta = -1.0 if K>S else 0.0
        gamma = 0.0
        return payoff, delta, gamma
    d1, d2 = _bs_d1_d2(S,K,r,q,sigma,T)
    disc_q = math.exp(-q*T)
    disc_r = math.exp(-r*T)
    if option=="call":
        val = disc_q*S*norm.cdf(d1) - disc_r*K*norm.cdf(d2)
        delta = disc_q*norm.cdf(d1)
    else:
        val = disc_r*K*norm.cdf(-d2) - disc_q*S*norm.cdf(-d1)
        delta = disc_q*(norm.cdf(d1) - 1.0)
    gamma = disc_q*norm.pdf(d1)/(S*sigma*math.sqrt(T))
    return val, delta, gamma


## CRR Tree with *Greeks on Grids* Δ, Γ, θ + CV Americans

In [ ]:

@dataclass
class Outputs:
    # BS European
    bs_euro_value: float
    bs_euro_delta: float
    bs_euro_gamma: float
    # Tree European (uncorrected)
    euro_value: float
    euro_delta: float
    euro_gamma: float
    euro_theta: float
    # Tree American (uncorrected)
    american_value: float
    american_delta: float
    american_gamma: float
    american_theta: float
    # CV-corrected Americans
    cv_value: float
    cv_delta: float
    cv_gamma: float
    cv_theta: float

def payoff(option: OptionType, S: np.ndarray, K: float) -> np.ndarray:
    return np.maximum(S-K,0.0) if option=="call" else np.maximum(K-S,0.0)

def crr_tree_greeks_on_grids(
    option: OptionType,
    S0: float, K: float, r: float, q: float, sigma: float, T: float,
    steps: int
) -> Outputs:
    if steps < 2:
        raise ValueError("Need at least 2 steps to compute grid-based Δ, Γ, θ (uses levels 0–2).")
    dt = T/steps
    u = math.exp(sigma*math.sqrt(dt))
    d = 1.0/u
    disc = math.exp(-r*dt)
    growth = math.exp((r - q)*dt)
    p = (growth - d)/(u - d)
    if not (0.0 <= p <= 1.0):
        raise ValueError("Risk-neutral probability out of bounds; adjust params/steps.")

    # Stock grid
    S_levels = [np.array([S0])]
    for n in range(1, steps+1):
        prev = S_levels[-1]
        cur = np.concatenate(([prev[0]*d], prev*u))
        S_levels.append(cur)

    # Terminal payoffs
    euro_vals = payoff(option, S_levels[-1], K)
    amer_vals = euro_vals.copy()
    euro_layers = [None]*(steps+1)
    amer_layers = [None]*(steps+1)
    euro_layers[-1] = euro_vals.copy()
    amer_layers[-1] = amer_vals.copy()

    # Backward induction
    for n in range(steps-1, -1, -1):
        euro_vals = disc*(p*euro_vals[1:] + (1-p)*euro_vals[:-1])
        amer_vals = disc*(p*amer_vals[1:] + (1-p)*amer_vals[:-1])
        amer_vals = np.maximum(amer_vals, payoff(option, S_levels[n], K))  # early exercise
        euro_layers[n] = euro_vals.copy()
        amer_layers[n] = amer_vals.copy()

    # Greeks on grids at start
    h = 0.5 * S0 * (u - d)                     # averaged spacing at level 0
    def grid_delta(layer1):                     # central diff over level-1 nodes
        return (layer1[1] - layer1[0])/(2.0*h)
    def grid_gamma(layer2):                     # central second diff over level-2 nodes
        up, mid, dn = layer2[2], layer2[1], layer2[0]
        return (up - 2.0*mid + dn)/(h*h)
    def grid_theta(layer0, layer2):             # forward time diff via level-2 center
        return (float(layer2[1]) - float(layer0[0]))/(2.0*dt)

    euro_value = float(euro_layers[0][0])
    american_value = float(amer_layers[0][0])
    euro_delta = grid_delta(euro_layers[1])
    american_delta = grid_delta(amer_layers[1])
    euro_gamma = grid_gamma(euro_layers[2])
    american_gamma = grid_gamma(amer_layers[2])
    euro_theta = grid_theta(euro_layers[0], euro_layers[2])
    american_theta = grid_theta(amer_layers[0], amer_layers[2])

    # Black–Scholes European
    bs_val, bs_d, bs_g = bs_value_delta_gamma(option, S0, K, r, q, sigma, T)

    # Control variate for Americans
    cv_value = american_value + (bs_val - euro_value)
    cv_delta = american_delta + (bs_d - euro_delta)
    cv_gamma = american_gamma + (bs_g - euro_gamma)
    # θ CV: apply the same constant level shift for layers 0 and 2 before differencing
    cv_theta = ((amer_layers[2][1] + (bs_val - euro_value)) - (amer_layers[0][0] + (bs_val - euro_value))) / (2.0*dt)

    return Outputs(
        bs_val, bs_d, bs_g,
        euro_value, euro_delta, euro_gamma, euro_theta,
        american_value, american_delta, american_gamma, american_theta,
        float(cv_value), float(cv_delta), float(cv_gamma), float(cv_theta)
    )


### Quick check

In [ ]:

_ = crr_tree_greeks_on_grids("put", 50, 50, 0.1, 0.02, 0.3, 0.1, steps=8)
print("OK")


## Study configuration

In [ ]:

S0, r, q, sigma, T = 50.0, 0.1, 0.02, 0.3, 0.1
strikes = [46.0, 53.0]   # puts
opt = "put"


## Proxy from 1000-step CV Americans (V, Δ, Γ)

In [ ]:

proxy = []
for K in strikes:
    o = crr_tree_greeks_on_grids(opt, S0, K, r, q, sigma, T, steps=1000)
    proxy.append({"Strike":K, "X_V":o.cv_value, "X_Delta":o.cv_delta, "X_Gamma":o.cv_gamma})
proxy_df = pd.DataFrame(proxy)
proxy_df


## Sanity snapshot (signs, American ≥ Euro, θ)

In [ ]:

def snap(K, N=1000):
    o = crr_tree_greeks_on_grids(opt, S0, K, r, q, sigma, T, steps=N)
    return {
        "Strike": K,
        "BS Euro": o.bs_euro_value,
        "Tree Euro": o.euro_value,
        "American": o.american_value,
        "CV American": o.cv_value,
        "EarlyEx Premium": o.american_value - o.euro_value,
        "CV Δ": o.cv_delta,
        "CV Γ": o.cv_gamma,
        "CV θ": o.cv_theta,
    }
pd.DataFrame([snap(K) for K in strikes])


## Big-tree errors for **uncorrected** Americans (N ∈ {100,105,110,120})

In [ ]:

bigNs = [100,105,110,120]
rows = []
for K in strikes:
    p = proxy_df.loc[proxy_df["Strike"]==K].squeeze()
    for N in bigNs:
        o = crr_tree_greeks_on_grids(opt, S0, K, r, q, sigma, T, steps=N)
        rows.append({
            "Strike":K,"Steps":N,
            "E_V":abs(o.american_value - p["X_V"]),
            "E_Delta":abs(o.american_delta - p["X_Delta"]),
            "E_Gamma":abs(o.american_gamma - p["X_Gamma"]),
        })
big_err_df = pd.DataFrame(rows)
avg_err = big_err_df.groupby("Strike")[["E_V","E_Delta","E_Gamma"]].mean().rename(
    columns={"E_V":"E_V_H","E_Delta":"E_Delta_H","E_Gamma":"E_Gamma_H"})
display(big_err_df)
avg_err


## Smallest N where **CV** meets big-tree average errors; savings factor

In [ ]:

mean_bigN = np.mean(bigNs)
searchN = range(5,121)

results = []
for K in strikes:
    thr = avg_err.loc[K]
    p = proxy_df.loc[proxy_df["Strike"]==K].squeeze()
    for metric, thr_key, p_key in [("Value","E_V_H","X_V"),("Delta","E_Delta_H","X_Delta"),("Gamma","E_Gamma_H","X_Gamma")]:
        target = thr[thr_key]
        bestN, bestErr = None, None
        for N in searchN:
            o = crr_tree_greeks_on_grids(opt, S0, K, r, q, sigma, T, steps=N)
            val = {"Value":o.cv_value, "Delta":o.cv_delta, "Gamma":o.cv_gamma}[metric]
            err = abs(val - p[p_key])
            if err <= target:
                bestN, bestErr = N, err
                break
        if bestN is None:
            bestN, bestErr = max(searchN), float("nan")
        results.append({
            "Strike":K, "Metric":metric,
            "Threshold":target, "Best_Steps":bestN,
            "CV_Error":bestErr, "Savings_Factor":mean_bigN/bestN
        })
results_df = pd.DataFrame(results)
results_df


## Compact error table (uncorrected vs CV)

In [ ]:

def error_table(K, Ns=(5,10,20,40,80,120)):
    p = proxy_df.loc[proxy_df["Strike"]==K].squeeze()
    out = []
    for N in Ns:
        o = crr_tree_greeks_on_grids(opt, S0, K, r, q, sigma, T, steps=N)
        out.append({
            "N":N,
            "uncorr_V":abs(o.american_value - p["X_V"]),
            "cv_V":abs(o.cv_value - p["X_V"]),
            "uncorr_D":abs(o.american_delta - p["X_Delta"]),
            "cv_D":abs(o.cv_delta - p["X_Delta"]),
            "uncorr_G":abs(o.american_gamma - p["X_Gamma"]),
            "cv_G":abs(o.cv_gamma - p["X_Gamma"]),
        })
    return pd.DataFrame(out)

err46 = error_table(46.0)
err53 = error_table(53.0)
display(err46)
err53
